## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,-9.8000,-139.0333,78.03,74,29,15.73,scattered clouds
1,1,Cap Malheureux,MU,-19.9842,57.6142,70.29,67,68,5.99,broken clouds
2,2,Garden City,US,42.3256,-83.3310,87.15,46,40,10.36,scattered clouds
3,3,Ponta Do Sol,PT,32.6667,-17.1000,75.70,83,31,2.28,scattered clouds
4,4,Mataura,NZ,-46.1927,168.8643,39.47,94,24,1.92,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?72
What is the maximum temperature you would like for your trip?78


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Ponta Do Sol,PT,32.6667,-17.1000,75.70,83,31,2.28,scattered clouds
9,9,Iwanai,JP,42.9744,140.5089,75.81,85,44,3.89,scattered clouds
13,13,Rikitea,PF,-23.1203,-134.9692,73.96,75,21,16.35,few clouds
15,15,Hasaki,JP,35.7333,140.8333,77.40,81,35,5.14,scattered clouds
24,24,Sapele,NG,5.8941,5.6767,74.16,94,78,3.00,broken clouds
25,25,Krasnoyarskiy,RU,51.9650,59.8967,75.27,46,85,10.56,overcast clouds
27,27,Pryyutivka,UA,48.7209,33.0708,73.18,48,51,10.49,broken clouds
48,48,Biak,ID,-0.9131,122.8766,75.61,89,88,8.63,overcast clouds
49,49,Salalah,OM,17.0151,54.0924,76.60,89,93,8.32,overcast clouds
69,69,Youkounkoun,GN,12.5333,-13.1333,76.23,90,99,5.59,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

WeatherPy_Database     122
City                   122
Country                122
Lat                    122
Lng                    122
Max Temp               122
Humidity               122
Cloudiness             122
Wind Speed             122
Current Description    122
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ponta Do Sol,PT,75.70,scattered clouds,32.6667,-17.1000,
9,Iwanai,JP,75.81,scattered clouds,42.9744,140.5089,
13,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,
15,Hasaki,JP,77.40,scattered clouds,35.7333,140.8333,
24,Sapele,NG,74.16,broken clouds,5.8941,5.6767,
25,Krasnoyarskiy,RU,75.27,overcast clouds,51.9650,59.8967,
27,Pryyutivka,UA,73.18,broken clouds,48.7209,33.0708,
48,Biak,ID,75.61,overcast clouds,-0.9131,122.8766,
49,Salalah,OM,76.60,overcast clouds,17.0151,54.0924,
69,Youkounkoun,GN,76.23,overcast clouds,12.5333,-13.1333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"]= f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url ="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping.") 
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [9]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ponta Do Sol,PT,75.70,scattered clouds,32.6667,-17.1000,Hotel do Campo
9,Iwanai,JP,75.81,scattered clouds,42.9744,140.5089,いわない高原ホテル
13,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,People ThankYou
15,Hasaki,JP,77.40,scattered clouds,35.7333,140.8333,Hotel Sunrise Choshi
24,Sapele,NG,74.16,broken clouds,5.8941,5.6767,Movig Hotel & Suites
25,Krasnoyarskiy,RU,75.27,overcast clouds,51.9650,59.8967,
27,Pryyutivka,UA,73.18,broken clouds,48.7209,33.0708,Многоквартирный Дом по адресу: Войновское шосе...
48,Biak,ID,75.61,overcast clouds,-0.9131,122.8766,Penginapan Kim
49,Salalah,OM,76.60,overcast clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
69,Youkounkoun,GN,76.23,overcast clouds,12.5333,-13.1333,


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != '']
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ponta Do Sol,PT,75.70,scattered clouds,32.6667,-17.1000,Hotel do Campo
9,Iwanai,JP,75.81,scattered clouds,42.9744,140.5089,いわない高原ホテル
13,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,People ThankYou
15,Hasaki,JP,77.40,scattered clouds,35.7333,140.8333,Hotel Sunrise Choshi
24,Sapele,NG,74.16,broken clouds,5.8941,5.6767,Movig Hotel & Suites
...,...,...,...,...,...,...,...
657,Maumere,ID,77.47,broken clouds,-8.6199,122.2111,Sylvia Hotel
667,Purwodadi,ID,73.60,broken clouds,-7.0868,110.9158,OYO 1620 Sion Family Residence
669,Tamandare,BR,73.63,scattered clouds,-8.7597,-35.1047,Pousada 100 Passos
672,Mudbidri,IN,76.62,broken clouds,13.0833,74.9833,Hotel Navami Comforts


In [15]:
clean_hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Ponta Do Sol,PT,75.70,scattered clouds,32.6667,-17.1000,Hotel do Campo
9,Iwanai,JP,75.81,scattered clouds,42.9744,140.5089,いわない高原ホテル
13,Rikitea,PF,73.96,few clouds,-23.1203,-134.9692,People ThankYou
15,Hasaki,JP,77.40,scattered clouds,35.7333,140.8333,Hotel Sunrise Choshi
24,Sapele,NG,74.16,broken clouds,5.8941,5.6767,Movig Hotel & Suites
27,Pryyutivka,UA,73.18,broken clouds,48.7209,33.0708,Многоквартирный Дом по адресу: Войновское шосе...
48,Biak,ID,75.61,overcast clouds,-0.9131,122.8766,Penginapan Kim
49,Salalah,OM,76.60,overcast clouds,17.0151,54.0924,HAMDAN PLAZA HOTEL SALALAH
80,Naze,JP,77.56,overcast clouds,28.3667,129.4833,Amami Port Tower Hotel
81,Amuntai,ID,73.56,overcast clouds,-2.4177,115.2494,Hotel Balqis


In [16]:
# 8a. Create the output File (CSV)
output_data_file ="WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                max_intensity=300, point_radius=4)

In [22]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))